In [1]:
from git_utils import get_repo_file_path
from kaggle_utils import descargar_dataset_kaggle


# Asegúrarse de que el archivo kaggle.json con las credenciales de kaggle
# estén en el directorio base del repositorio.
kaggle_credentials_path = get_repo_file_path("kaggle.json")

descargar_dataset_kaggle(
    "https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data",
    kaggle_credentials_path,
    "../data/",
)

Dataset URL: https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data


100%|██████████| 262M/262M [00:33<00:00, 8.31MB/s] 
